In [ ]:
import json
import random
import re
import subprocess
import time

import config as c
import requests

In [ ]:
# login to Transkribus
my_session = requests.Session()
my_session.post(
    "https://transkribus.eu/TrpServer/rest/auth/login",
    data={"user": c.login, "pw": c.password},
)

In [ ]:
# set collection id
collid = "72436"

In [ ]:
# get docids and sesamids from Transkribus
docs = my_session.get(
    "https://transkribus.eu/TrpServer/rest/collections/{}/list".format(collid)
)

# only keep those records containing sesamid (regex check)
filtered_docs = [
    o for o in json.loads(docs.content) if re.findall(r"([a-fA-F\d]{32})", o["title"])
]

# extract ids
docids = [o["docId"] for o in filtered_docs if len(o["title"]) == 32]

Download Saxon HE, supports XSLT 2.0.
https://www.saxonica.com/download/java.xml

Unzip and insert the location below:

In [ ]:
saxonLocation = "/home/magbb/programs/saxon/saxon-he-10.3.jar"

In [ ]:
for docid in docids:
    print(docid)
    doc = my_session.get(
        "https://transkribus.eu/TrpServer/rest/collections/{}/{}/fulldoc".format(
            collid, docid
        )
    )

    docRows = []

    for page in json.loads(doc.content)["pageList"]["pages"]:
        pageNr = page["pageNr"]
        print("--", pageNr)
        page = my_session.get(
            "https://transkribus.eu/TrpServer/rest/collections/{}/{}/{}/text".format(
                collid, docid, pageNr
            )
        )

        pagexmlName = (
            "PAGEXML"
            + "_"
            + str(collid)
            + "_"
            + str(docid)
            + "_"
            + str(pageNr)
            + ".xml"
        )
        altoxmlName = (
            "ALTOXML"
            + "_"
            + str(collid)
            + "_"
            + str(docid)
            + "_"
            + str(pageNr)
            + ".xml"
        )

        with open("docs/" + pagexmlName, "wb") as f:
            f.write(page.content)

        # transform
        subprocess.call(
            [
                "java",
                "-jar",
                saxonLocation,
                "-o:docs/%s" % (altoxmlName),
                "-s:docs/%s" % (pagexmlName),
                "xslt/PageToAltoWordLevel.xsl",
            ]
        )

        time.sleep(random.randint(0, 1))